In [67]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime

In [68]:
def get_spotify_data(url, headers):
  response = requests.get(url = url, headers = headers)
  if response.status_code == 200:
    response.encoding = 'utf-8'
  else:
    print("Error connection")
  spotify_html = response.text
  spotify_bs4 = BeautifulSoup(spotify_html,parser = "html.parser")
  spotify_data = spotify_bs4.find_all("table",class_ = "sortable")[0].find_all("tr")
  return spotify_data

In [69]:
def get_columns(spotify_data):
  columns = []
  for item in spotify_data[0].find_all("th"):
    columns.append(item.text)
  # print(columns)

  # 調整PK,x,Artist,Title
  index_pk = columns.index("Pk")
  index_a_t = columns.index("Artist and Title")

  columns[index_pk+1] = "x"
  columns = ["Artist","Title"]+ columns[index_a_t+1:]

  return columns, index_pk, index_a_t

In [ ]:
def get_rows(spotify_data, index_pk, index_a_t):
  rows = []
  for item in spotify_data[1:]:
    # 分行
    all_elements = item.text.split("\n")
    # 去空字串
    elements_not_empty = [e for e in all_elements if e != '']
    # 分開歌手與歌名
    artist_and_title = result = re.split(r'\s-\s', elements_not_empty[index_a_t])

    artist = artist_and_title[0]
    title = artist_and_title[-1]
    # 分開pk 和 x (peak和?)
    pk = int(elements_not_empty[index_pk].split("(")[0].replace(",",""))
    try:
      x = int(elements_not_empty[index_pk].split("(")[1][1:-1])
    except:
      x = ""
    # 組合成一列
    r1 = [artist,title]
    r2 = [int(e.replace(",","")) for e in elements_not_empty[index_a_t+1:index_pk]]
    r3 = [pk,x]
    r4 = [int(e.replace(",","")) for e in elements_not_empty[index_pk+1:]]
    row = r1+r2+r3+r4

    rows.append(row)

  return rows

In [ ]:
def to_csv(df,web_page_name):
  standard_year = datetime.now().date().year
  standard_month = datetime.now().date().month
  standard_day = datetime.now().date().day
  standard_hour = datetime.now().hour
  standard_minute = datetime.now().minute
  file_name = \
  f"{web_page_name}_{standard_year}_{standard_month:02d}_{standard_day:02d}"+ \
  f"_{standard_hour:02d}{standard_minute:02d}.csv"
  df.to_csv(file_name,encoding = "utf-8")

In [73]:
country_codes = ["TW","US"]
time_spans = ["daily","weekly"]
# total : data 2014_8_10-today
total = False #True
headers ={
    "User-Agent": \
    "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) \
    AppleWebKit/537.36 (KHTML, like Gecko) \
    Chrome/127.0.0.0 Mobile Safari/537.36"
}

In [72]:
for country in [name.lower() for name in country_codes]:
  for time_span in time_spans:
    web_page_name = f"{country}_{time_span}{'_totals'*total}"
    url = f"https://kworb.net/spotify/country/{web_page_name}.html"
    spotify_data = get_spotify_data(url, headers)
    # print(columns)
    columns, *index = get_columns(spotify_data)
    rows = get_rows(spotify_data, *index)
    # print(rows)
    df = pd.DataFrame(rows,columns = columns)
    # print("country")
    # display(df.head())
    to_csv(df,web_page_name)